# 1. Requests

## 1.1 package install
- `$ pip install requests`
- `$ pip install bs4`
- `$ pip install python-forecastio`

In [1]:
import requests

- api json을 사용 : 직방, 네이버 주식 데이터 크롤링
- api json을 사용, forecastio : dark sky api로 날씨 데이터를 수집
- bs4 : 네이버 실시간 키워드, 다음 실시간 키워드 크롤링
- web file url로 파일을 다운로드 받는 방법

## 1.2 API를 이용하여 JSON 파싱

### 1.2.1 네이버 주식 데이터 가져오기
- https://m.stock.naver.com/

In [2]:
# url 만들기
def make_url(pageSize=10, page=1):
    return "http://m.stock.naver.com/api/json/sise/siseListJson.nhn?\
menu=market_sum&sosok=0&pageSize=" + str(pageSize) + "&page=" + str(page)
make_url()

'http://m.stock.naver.com/api/json/sise/siseListJson.nhn?menu=market_sum&sosok=0&pageSize=10&page=1'

In [3]:
# 데이터를 가져와서 판다스의 데이터프레임으로 만드는 함수
def get_stock_data_1(url):
    response = requests.get(url)
    json_info = response.json()
    companys = json_info["result"]["itemList"]
    df = pd.DataFrame(columns=["종목","시세","전일비","등락율","시가총액","거래량"])
    for company in companys:
        df.loc[len(df)] = {
            "종목":company["nm"],
            "시세":company["nv"],
            "전일비":company["cv"],
            "등락율":company["cr"],
            "시가총액":company["mks"],
            "거래량":company["aq"],
        }
    return df

In [4]:
# json_normalize 를 사용하면 parsing이 더 편합니다.
from pandas.io.json import json_normalize
def get_stock_data_2(url):
    response = requests.get(url)
    json_info = response.json()
    companys = json_info["result"]["itemList"]
    return json_normalize(companys)

In [5]:
url = make_url(2000, 1)
df = get_stock_data_2(url)
df.head()

Empty DataFrame
Columns: []
Index: []

### 1.2.2 dark sky api
- 위도와 경도를 입력하면 날씨 정보를 보내주는 api
- https://api.darksky.net/

- 토큰 획득

In [6]:
FORECAST_TOKEN = "56774bd2dd9b4f0320516e978bf778db"

In [7]:
def forecast(lat, lng):
    url = "https://api.darksky.net/forecast/{}/{},{}".format(FORECAST_TOKEN, lat, lng)
    response = requests.get(url)
    json_info = response.json()
    return json_info["timezone"], json_info["hourly"]["summary"]

In [8]:
lat = 37.512
lng = 126.954
forecast(lat, lng)

('Asia/Seoul', 'Partly cloudy starting this afternoon.')

- 모듈을 이용

In [9]:
import forecastio
def forecast2(lat, lng):
    forecast = forecastio.load_forecast(FORECAST_TOKEN, lat, lng)
    byHourly = forecast.hourly()
    return byHourly.summary

In [10]:
lat = 37.512
lng = 126.954
forecast2(lat, lng)

'Partly cloudy starting this afternoon.'

## 1.3 bs4을 이용하여 파싱
- 네이버 키워드 랭킹 데이터 크롤링
- 다음 키워드 랭킹 데이터 크롤링

### 1.3.1 네이버 실시간 키워드 크롤링

In [11]:
from bs4 import BeautifulSoup

In [12]:
# dom.select - 여러개의 html 엘리먼트를 셀렉팅 할때 사용, 결과로 리스트 데이터를 리턴
# dom.select_one - 하나의 html 엘리먼트를 셀렉팅 할때 사용, 결과로 문자열 데이터를 리턴

def naver():
    df = pd.DataFrame(columns=["rank","keyword"])
    
    response = requests.get("https://www.naver.com/")
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select(".ah_roll_area > .ah_l > .ah_item")
    
    for keyword in keywords:
        df.loc[len(df)] = {
            "rank": keyword.select_one(".ah_r").text,
            "keyword": keyword.select_one(".ah_k").text,
        }
        
    return df
    
naver_df = naver()    
naver_df.head()

rank     keyword
0    1        택시파업
1    2      카카오 카풀
2    3  강서구 pc방 살인
3    4         정유미
4    5         나영석

### 1.3.2 Daum 실시간 키워드 크롤링

In [13]:
def daum():
    df = pd.DataFrame()
    response = requests.get("https://www.daum.net")
    dom = BeautifulSoup(response.content, 'html.parser')
    keywords = dom.select(".realtime_part .list_hotissue.issue_row > li")
    rank = [ keyword.select_one(".ir_wa").text[:-1] for keyword in keywords ]
    keywords = [ keyword.select_one(".link_issue").text for keyword in keywords ]
    df["rank"] = rank
    df["keyword"] = keywords
    return df

daum_df = daum()
daum_df.head()

rank keyword
0    1  카카오 카풀
1    2     김지수
2    3   택시 파업
3    4     이필모
4    5    우드러프

### 1.3.3 공통 키워드 추출

In [14]:
result = [
    keyword 
    for keyword in naver_df["keyword"] 
    if daum_df["keyword"].str.contains(keyword).any()
]
result

['카카오 카풀', '김지수', '택시', '카풀', '난민', '이필모', '우드러프']

In [15]:
set(naver_df["keyword"]) & set(daum_df["keyword"])

{'김지수', '난민', '우드러프', '이필모', '카카오 카풀'}

### 1.4 직방 사이트 api를 이용하여 크롤링
- 동을 입력하면 해당 동의 매물 정보를 크롤링하여 데이터 프레임으로 만듦
- 크롤링하는 순서
    - 지역정보 - 위도, 경도
    - 아이템 리스트 - 매물 아이디 리스트
    - 매물 정보 가져오기

#### 1.4.1 지역정보 가져오기
- 동을 입력하면 (시, 구, 동)과 (위도, 경도)값을 출력

In [16]:
def get_addr_info(keyword):
    url = "https://apis.zigbang.com/search/all?q=" + keyword + "&type=oneroom";
    response = requests.get(url)
    json_info = response.json()
    return json_info["items"][0]

In [17]:
result = get_addr_info("망원동")
result['description'], result['lat'], result['lng']

('서울시 마포구 망원동', 37.556785583496094, 126.9013442993164)

#### 1.4.2 아이템 리스트 검색
- 위도 경도 0.01 차이로 검색

In [18]:
from urllib.parse import urlencode
def get_item_numbers(lat, lng):
    params = {
        "lat_south": lat - 0.01,
        "lat_north": lat + 0.01,
        "lng_west": lng - 0.01,
        "lng_east": lng + 0.01,
        "room_types": "[01,02,03,04,05]",
    }
    params_str = urlencode(params)
    url = "https://api.zigbang.com/v3/items2?" + params_str
    response = requests.get(url)
    json_info = response.json()
    items = json_info["list_items"]
    result = [item["simple_item"]["item_id"] for item in items]
    return result

In [19]:
item_ids = get_item_numbers(result['lat'], result['lng'])
len(item_ids), item_ids[:5]

(428, [13043291, 12688736, 13048854, 12984226, 12956569])

#### 1.4.3 매물정보 가져오기

In [20]:
def get_items(item_ids):
    url = "https://api.zigbang.com/v3/items?detail=true&item_ids=" + (str(item_ids)).replace(" ","")
    response = requests.get(url)
    json_info = response.json()
    items = json_info["items"]
    datas = [item["item"] for item in items]
    result_df = json_normalize(datas)
    filter_columns = ["rent", "deposit", "floor", "size", "address1", "options", "manage_cost", "parking", "elevator", "movein_date",\
                  "agent_name", "agent_phone", "agent_email"]
    return result_df[filter_columns]

In [21]:
result_df = get_items(item_ids[:])
result_df.tail()

rent  deposit   floor  size     address1  \
423     0    30000      4층  11.6  서울시 마포구 망원동   
424    65     1000  저층/13층   7.0  서울시 마포구 성산동   
425     0    21000      2층  18.0  서울시 마포구 망원동   
426     0    16000  고층/12층   5.0  서울시 마포구 성산동   
427    55     1000  고층/16층   5.6  서울시 마포구 성산동   

                                    options manage_cost parking elevator  \
423                             에어컨,신발장,싱크대         5만원     불가능       없음   
424                                       -         6만원     불가능       있음   
425                                       -          없음     불가능       없음   
426  에어컨,냉장고,세탁기,인덕션,전자레인지,책상,책장,옷장,신발장,싱크대         6만원      가능       있음   
427        에어컨,냉장고,세탁기,인덕션,책상,책장,옷장,신발장,싱크대         6만원      가능       있음   

    movein_date        agent_name   agent_phone          agent_email  
423        즉시입주      우정공인중개사(김현하)   02-704-8949  cookdog369@nate.com  
424        즉시입주   두산센티움공인중개사(김윤희)  02-3152-8982  eun6509@hanmail.net  
425        날짜협의      연오공인중개사(이승우)   02-338-7778    mysoung@naver.com  
426   11월 이후 협의  시티플레이스공인중개사(이동호)   02-921-9557    ctplace@naver.com  
427     10월날짜가능    상암명가공인중개사(고성란)   02-304-2280   rrsk57@hanmail.net

In [22]:
def main(addr):
    add_info = get_addr_info(addr)
    item_ids = get_item_numbers(add_info['lat'], add_info['lng'])
    return get_items(item_ids)

In [23]:
df = main("망원동")
df.tail()

rent  deposit floor  size     address1                       options  \
423     0    17500    2층  11.0  서울시 마포구 합정동                 가스레인지,신발장,싱크대   
424    40      500   반지하  11.0  서울시 마포구 망원동                             -   
425     0    13000    3층   8.0  서울시 마포구 망원동  에어컨,냉장고,세탁기,가스레인지,옷장,신발장,싱크대   
426    45      300    3층   5.0  서울시 마포구 망원동    에어컨,냉장고,세탁기,인덕션,옷장,신발장,싱크대   
427    40      500    3층   5.0  서울시 마포구 망원동            에어컨,냉장고,세탁기,인덕션,옷장   

    manage_cost parking elevator movein_date        agent_name  agent_phone  \
423         2만원      가능       없음        입주협의      세하공인중개사(박경희)  02-323-5669   
424         1만원     불가능       없음        즉시입주      조은공인중개사(유광선)  02-323-8954   
425         3만원      가능       없음        즉시입주     성미산공인중개사(추선중)  02-335-7232   
426         5만원     불가능       없음    즉시입주가능!!  홍대역부동산공인중개사(최동명)  02-324-1800   
427         5만원     불가능       없음        즉시입주      이화공인중개사(하철주)  02-333-5353   

               agent_email  
423      a941010@naver.com  
424      haroo99@naver.com  
425     1014sunny@daum.net  
426   dkdlemf222@naver.com  
427  gladiac99@hanmail.net